# ENGN2350 Data-Driven Design and Analysis of Structures and Materials

_Homeworks for fall semester 2025-2026_

Coding exercises to explore the [`f3dasm`](https://f3dasm.readthedocs.io/en/latest/) package.

**General instructions**:

- Read the questions and answer in the cells under the "PUT YOUR CODE IN THE CELL BELOW" message.
- Work through the notebook and make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. You can remove the `'raise NotImplementedError()'` code.
- After "END OF YOUR CODE" , there is a cell that contains simple tests (with `assert` statements) to see if you did the exercises correctly. Not all exercises have tests! If you run the cell containing the tests and no error is given, you have succesfully solved the exercise!
- Make sure you have the right version of `f3dasm` (2.1.0).

> You can check your `f3dasm` version by running `pip show f3dasm`

- **ONLY WORK ON THE EXERCISE IN A JUPYTER NOTEBOOK ENVIRONMENT**

> The homework assignments are generated and automatically graded by the `nbgrader` extension. If you open and save the notebook in Google Colab, metadata from Colab will be added, and the `nbgrader` metadata will be altered. As a result, `nbgrader` will be unable to automatically grade your homework. Therefore, we kindly ask students to only work on the notebook in Jupyter Notebook.

- **DO NOT ADD OR REMOVE CELLS IN THE NOTEBOOK**

> Most cells containing tests are set to read-only, but VS Code can bypass this restriction. Modifying or removing cells in the notebook may disrupt the `nbgrader` system, preventing automatic grading of your homework.

**Instructions for handing in the homework**

- Upload the Jupyter Notebook (`.ipynb file`) to Canvas

If there are any questions about the homework, send an email to Samik (samik_mukhopadhyay@brown.edu) or Elvis (elvis_alexander_aguero_vera@brown.edu)

**Grading**

- In each homework, you can obtain a maximum of 20 points
- Next to each subquestion, the maximum amount of obtainable points is listed

Good luck!

You can put your name in the cell below:

In [ ]:
NAME = ""

---

## Homework 9

In this homework, you will delve into optimization using the `f3dasm` package.

By the end of this homework, you will:
- Learn how to implement 3rd party libraries into the `f3dasm` framework
- Gain a deeper understanding of the strengths and weaknesses of various optimization algorithms.
- Learn how initial conditions and hyperparameters affect optimization performance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import f3dasm

---

### Exercise 1

`f3dasm` comes with only a few built-in functionalities - and that is intentional. Its main purpose is to provide interface classes, giving you the flexibility to implement your own algorithms and use them just like the defaults. If you’re already relying on third-party optimizers in your code, you can easily integrate them with f3dasm. In the following exercise, we’ll show you how to create an interface between `f3dasm` and a third-party implementation, so you can take advantage of the `f3dasm` interface in your workflow.

In this case study, we focus on implementing the Covariance Matrix Adaptation Evolution Strategy (CMA-ES), a state-of-the-art gradient-free optimizer [[Hansen, 1996]](https://ieeexplore.ieee.org/abstract/document/542381?casa_token=t4_3OFlyxSMAAAAA:ekh3JAbGpzFA1hWkzA8jG0zyCh1AG_11EEpehwzG-CjFHqlkhV2F35SILK_rcxlHdUenAQ). Rather than coding the algorithm from scratch, we take advantage of an existing Python implementation provided by the [cmaes](https://github.com/CyberAgentAILab/cmaes) library. Our goal is to integrate this optimizer into the `f3dasm` framework.

First we make sure to install the `cmaes` package from pip:

In [ ]:
!pip install cmaes==0.12.0

The README.md file of the GitHub repository gives you an idea how the library is used natively:

```python
import numpy as np
from cmaes import CMA

def quadratic(x1, x2):
    return (x1 - 3) ** 2 + (10 * (x2 + 2)) ** 2

if __name__ == "__main__":
    optimizer = CMA(mean=np.zeros(2), sigma=1.3) # create the optimizer instance

    for generation in range(50): # run for 50 iterations
        solutions = [] # initialize empty list of solutions
        for _ in range(optimizer.population_size): # run for each individual in the population
            x = optimizer.ask() # ask the optimizer for the next query point
            value = quadratic(x[0], x[1]) # evaluate the function (= data generation!)
            solutions.append((x, value)) # add the solution to the list
            print(f"#{generation} {value} (x1={x[0]}, x2 = {x[1]})")
        optimizer.tell(solutions) # update the optimizer state by telling the newly acuired population
```


In order to make this compatible with `f3dasm`, we need to create class inheriting from `f3dasm.Optimizer` where we implement the `arm` and `call` method:

```python
class CMAESOptimizer(f3dasm.Optimizer):
    def arm(self, data: ExperimentData, data_generator: DataGenerator, input_name: str, output_name: str) -> None:
        ...
        
    def call(self, data: ExperimentData, n_iterations: int, **kwargs) -> ExperimentData:
        ...
```

Notice that in order to initialize `optimizer`, we need to provide the shape of the input parameter.
We can use the `data` argument provided by the `arm` call to to select the tunable parameter (`input_name`) in our `Domain` and inspect it's shape.
Additionally, we can set our output parameter to optimizer (`output_name`).
In the `call` method, we should run the CMA-ES update step iteratively. To make the code more readable, we can create a private `_step` method that only computes one iteration and call this `_step` function `n_iterations` number of times.

---

1.1 _(4 points)_ Fill in the missing code snippets in the `CMAESOptimizer` class below:

`arm`
- Extract the selected input parameter from the domain in the `arm` method
- Define the bounds array (shape= (2, dimensionality)) for the `CMA` class
- Initialize the CMA-ES optimizer

`_step`
- Evaluate the samples with the self.data_generator

`call`
- Run the optimizer for n_iterations steps

YOUR CODE IN THE CELL BELOW

In [ ]:
# You might want to import some things here ..
# YOUR CODE HERE
raise NotImplementedError()
from f3dasm import Optimizer, ExperimentData, DataGenerator
from f3dasm.design import Domain

class CMAESOptimizer(Optimizer):
    def __init__(self, seed: int, sigma: float = 1.0):
        """
        CMA-ES optimizer.
        
        Parameters
        ----------
        seed : int
            Random seed for reproducibility.
        sigma : float
            Initial standard deviation of the covariance matrix, default 1.0 
        """
        self.seed = seed
        self.sigma = sigma

    def arm(self, data: ExperimentData, data_generator: DataGenerator,
            input_name: str, output_name: str):
        """
        Prepare the CMA-ES optimizer with the given data generator and parameters.
        
        Parameters
        ----------
        data : ExperimentData
            The experiment data containing the domain information.
        data_generator : DataGenerator
            The data generator to evaluate the samples.
        input_name : str
            The name of the input parameter to optimize.
        output_name : str
            The name of the output parameter to optimize.
        """
        self.data_generator = data_generator
        self.output_name = output_name
        self.input_name = input_name

        # Extract the selected input parameter from the domain
        input_parameter = ...
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # Define the bounds array (shape= (2, dimensionality)) for CMA-ES
        bounds = ...
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # Initialize the CMA-ES optimizer
        self.optimizer = ...
        # YOUR CODE HERE
        raise NotImplementedError()

    def _step(self, domain: Domain):
        """
        Perform a single optimization step using CMA-ES.
        Parameters
        ----------
        domain : Domain
            The domain of the experiment.
        
        Returns
        -------
        ExperimentData
            The experiment data after performing one optimization step.
        """
        individuals = []
        for individual in range(self.optimizer.population_size):
            x = self.optimizer.ask()
            individuals.append({self.input_name: x})

        experiment_data = ExperimentData(input_data=individuals, domain=domain)

        # Evaluate the samples with the self.data_generator
        # YOUR CODE HERE
        raise NotImplementedError()
        
        samples = [
            (experiment_sample.input_data[self.input_name],
          experiment_sample.output_data[self.output_name])
          for idx, experiment_sample in experiment_data
          ]
        
        self.optimizer.tell(samples)

        return experiment_data

    def call(self, data: ExperimentData, n_iterations: int, **kwargs):
        """
        Run the CMA-ES optimization for a specified number of iterations.
        
        Parameters
        ----------
        data : ExperimentData
            The initial experiment data.
        n_iterations : int
            The number of optimization iterations to perform.
        **kwargs
            Additional keyword arguments (not used).
        Returns
        -------
        ExperimentData
            The accumulated experiment data after all optimization iterations.
        """
        # Create a new ExperimentData object
        solutions = ExperimentData(domain=data.domain)

        # Run the optimizer for n_iterations steps
        # YOUR CODE HERE
        raise NotImplementedError()
        return solutions


END OF YOUR CODE

---

1.2 _(2 points)_ Optimize the a 10D Levy function with your CMAES optimizer for $50$ iterations within the bounds $x \in [-10, 10]^d$. Use $123$ as the random seed and leave the initial standard deviation of the covariance matrix as $1.0$

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

1.3 _(1 points)_ Plot the objective value versus the number of function evaluations

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

1.3 _(1 point)_ Repeat exercise 1.2, but now use the built-in L-BFGS-B optimizer. Plot the objective value versus number of function evaluations for both optimizers in one plot.

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

### Exercise 2

Now we are going to compare different optimization techniques on different benchmark functions. You have implemented the gradient-free optimizer CMAES in the previous exercise. The other optimizer is the gradient-descent optimizer Adam. In the code block below, I have implemented this optimizer with numpy. The gradients are calculated using finiste differences:


In [ ]:
# Finite difference gradient
def numerical_grad(f, x, eps=1e-8):
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x_pos = x.copy()
        x_neg = x.copy()
        x_pos[i] += eps
        x_neg[i] -= eps
        grad[i] = (f(x_pos) - f(x_neg)) / (2 * eps)
    return grad

class AdamOptimizer(Optimizer):
    def __init__(self, learning_rate: float = 1e-3, beta1: float = 0.9, beta2 = 0.999, eps: float = 1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps

    def arm(self, data: ExperimentData, data_generator: DataGenerator,
            input_name: str, output_name: str):
        self.data_generator = data_generator
        self.input_name = input_name
        self.output_name = output_name

        # Extract the last ExperimentSample as the starting point of optimization
        experiment_sample = data.get_experiment_sample(data.index[-1])
        self.x = experiment_sample.input_data[input_name]

        # Initialize the momentum term and the second moment estimate
        self.m = np.zeros_like(self.x)
        self.v = np.zeros_like(self.x)

        # Initialize the timestep
        self.t = 1

    def _step(self, domain: Domain):
        g = numerical_grad(self.data_generator.f, self.x)  # finite-difference gradient
        
        # Update biased first moment estimate
        self.m = self.beta1 * self.m + (1 - self.beta1) * g
        
        # Update biased second raw moment estimate
        self.v = self.beta2 * self.v + (1 - self.beta2) * (g**2)
        
        # Bias correction
        m_hat = self.m / (1 - self.beta1**self.t)
        v_hat = self.v / (1 - self.beta2**self.t)
        
        # Update parameters
        self.x -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.eps)

        # Update the timestep
        self.t += 1

        experiment_data = ExperimentData(input_data=[{self.input_name: self.x}], domain=domain)
        self.data_generator.arm(experiment_data)
        experiment_data = self.data_generator.call(experiment_data)

        return experiment_data
    
    def call(self, data: ExperimentData, n_iterations: int, **kwargs):
        """
        Run the CMA-ES optimization for a specified number of iterations.
        
        Parameters
        ----------
        data : ExperimentData
            The initial experiment data.
        n_iterations : int
            The number of optimization iterations to perform.
        **kwargs
            Additional keyword arguments (not used).
        Returns
        -------
        ExperimentData
            The accumulated experiment data after all optimization iterations.
        """
        # Create a new ExperimentData object
        solutions = ExperimentData(domain=data.domain)

        # Run the optimizer for n_iterations steps
        for epoch in range(n_iterations):
            solutions += self._step(data.domain)
        return solutions

---

We are going to evaluate the performance of twi optimizers on various benchmark functions. In order to do this, we construct a function that initializes a benchmark function with a given dimensionality and optimizes it with the requested optimizer:

In [ ]:
from typing import Optional
from f3dasm import create_sampler

def optimize_benchmark_fn(benchmark_fn: str, optimizer: str, dimensionality: int, seed: int, n_iterations: int,
                          lower_bound: float = 0.0, upper_bound: float = 1.0,
                          hyperparameters: Optional[dict] = None) -> ExperimentData:

    # If no overwriting hyperparameters are given, create an empty dictionary
    if hyperparameters is None:
        hyperparameters = {}

    # Create a single-objective, continuous domain with bounds [0., 1.]
    domain = Domain()
    domain.add_array(name='x', shape=(dimensionality,), low=lower_bound, high=upper_bound)
    domain.add_output(name='y')

    # Create an empty ExperimentData object from the domain
    data = ExperimentData(domain=domain)

    # Randomly sample an initial solution
    sampler = create_sampler('random', seed=seed)
    data = sampler.call(data, n_samples=1)

    # Create the benchmark function
    data_generator = create_datagenerator(benchmark_fn, output_names=['y'])

    # Evaluate the sample
    data = data_generator.call(data)

    # Create the optimizer

    if optimizer.lower() == 'adam':
        optimizer = AdamOptimizer(**hyperparameters)
    elif optimizer.lower() == 'cmaes':
        optimizer = CMAESOptimizer(**hyperparameters, seed=seed)
    else:
        optimizer = create_optimizer(optimizer, **hyperparameters)
        
    # optimizer = create_optimizer(optimizer, **hyperparameters, seed=seed)
    optimizer.arm(data, data_generator, input_name='x', output_name='y')

    # Optimize the function
    data += optimizer.call(data=data, n_iterations=n_iterations)

    # Return the data
    return data

The function takes the following inputs:

- **`benchmark_fn`**: The name of a [built-in benchmark function](https://f3dasm.readthedocs.io/en/latest/rst_doc_files/defaults.html#implemented-benchmark-functions) from `f3dasm`.
- **`optimizer`**: The name of a [built-in optimizer](https://f3dasm.readthedocs.io/en/latest/rst_doc_files/defaults.html#implemented-optimizers).
- **`dimensionality`**: The number of continuous input dimensions.
- **`seed`**: The seed for generating initial candidate solutions (this is done with random uniform sampling).
- **`n_iterations`**: The total number of function evaluations allowed for optimizing the function.
- **`lower_bound`**: The lower box-constraint of the feasible area, by default $0.0$
- **`upper_bound`**: The upper box-constraint of the feasible area, by default $1.0$
- **`hyperparameters`**: _(optional)_ A dictionary to set non-default hyperparameters for the optimizer.

The function tries to minimize the benchmark function $f(\vec{x})$ with a finite budget while changing the input $\vec{x} = (x^0, x^1, x^2 \dots x^D)$ with $D$ the dimensionality of the search space. We constrain $\vec{x}$ to be in the domain, $[low, high]^D$, 

The output of the `optimize_benchmark_fn` is an `ExperimentData` object containing the entire history of function evaluations during optimization.

---

2.1 _(1 points)_ Optimize the noiseless 2D `'Sphere'` function with the `'Adam'` optimizer with a learning rate of $10^{-3}$ for $500$ iterations by using the `optimize_benchmark_fn` in the domain $\vec{x} \in [-1.0, 1.0]^D$. Use $123$ as the random seed. 

Make a plot where you show the progression of the objective values (y-axis) w.r.t the number of function evaluations (x-axis).

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

In addition to tracking how the objective function value evolves with each update step, visualizing the input parameters throughout the iterations can provide valuable insights. This visualization, referred to as the optimizer's trajectory, can be achieved by plotting it over a contour plot of the benchmark function.

A **contour plot** is a two-dimensional visualization of a three-dimensional surface. In this plot, we represent constant values of the function on a 2D plane using contour lines, or "level curves." Each line on the plot connects points where the function value is the same. This allows us to easily see how the function changes over a domain, highlighting areas of increase and decrease.

In optimization, contour plots are especially useful for understanding the shape of the function landscape, which can reveal minima, maxima, and saddle points.

We can generate a contour plot of benchmark functions using the following function:

In [ ]:
from itertools import product

def plot_function(data_generator: str, resolution: int = 31, lower_bound: float = 0.0, upper_bound: float = 1.0):
    # Step 1: Create the input data by creating a grid of points, equally spaced
    points = np.linspace(lower_bound, upper_bound, resolution)
    
    # 2D grid (meshgrid)
    x, y = np.meshgrid(points, points)
    grid = np.column_stack([x.ravel(), y.ravel()])
    input_data = [{'x': xi} for xi in grid]
    
    # Step 2: Create the experiment data
    domain = Domain()
    domain.add_array('x', shape=(2,), low=lower_bound, high=upper_bound) 
    experiment_data = ExperimentData(input_data=input_data, domain=domain)
    
    # Step 3: Evaluate the grid of points on the data generator
    data_generator = create_datagenerator(data_generator, output_names='y')
    experiment_data = data_generator.call(data=experiment_data)
    
    # Step 3.1: Retrieve the best found point in the grid, estimating the 'global minimum'
    x_min = np.array([es.input_data['x'] for _, es in experiment_data.get_n_best_output(1)])
    
    array_in = np.stack([es.input_data['x'] for _, es in experiment_data])
    array_out = np.stack([es.output_data['y'] for _, es in experiment_data])
    
    # Step 4: Create the contour plot
    X = array_in[:, 0].reshape(resolution, resolution) # Reshape in order to make the contour plot
    Y = array_in[:, 1].reshape(resolution, resolution) # Reshape in order to make the contour plot
    Z = array_out.reshape(resolution, resolution)
    fig, ax = plt.subplots(figsize=(8, 6))
    contour = ax.contour(X, Y, Z, levels=50, cmap='viridis', zorder=0)
    ax.contourf(X, Y, Z, levels=50, cmap='viridis', alpha=0.7, zorder=0)
    ax.scatter(x_min[:, 0], x_min[:, 1], color='k', marker='x', label='global minimum') # Display the global minimum estimate with a black cross
    fig.colorbar(contour)
    ax.set_xlabel("$x_0$")
    ax.set_ylabel("$x_1$")
    return fig, ax

The function takes the following inputs:

- **`benchmark_fn`**: The name of a [built-in benchmark function](https://f3dasm.readthedocs.io/en/latest/rst_doc_files/defaults.html#implemented-benchmark-functions) from `f3dasm`.
- **`resolution`**: _(optional)_ The number of equally spaced points per dimension used for constructing the contour plot, by default $31$.
- **`lower_bound`**: The lower box-constraint of the feasible area, by default $0.0$
- **`upper_bound`**: The upper box-constraint of the feasible area, by default $1.0$

The function returns a Matplotlib figure (`plt.Figure`) and axis (`plt.Axes`) object, allowing you to use the axis to overlay the optimization trajectory.

---

2.2 _(1 points)_ Create a contour plot of the 'Sphere' function using the `plot_function` function and plot the trajectory (= 2D input dimension) of the Adam optimizer created in the previous exercise on top of this landscape.

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

2.3 _(2 points)_ Optimize the same benchmark function in exercise 2.1, but now use the Adam optimizer with **4 different values of the learning rate hyperparameter**: $10^{-3}$, $10^{-2}$, $10^{-1}$ and $10^{0}$. Use the same seed $123$ and run for a maximum of $500$ function evaluations.
- Create a contour plot with the trajectories of the four Adam optimizers
- Make one figure where you plot the objective value w.r.t. the number of function evaluations for the four runs.
---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

2.4 _(1 point)_ Answer the following conceptual question:

- What does the learning rate hyperparameter control for the Adam optimizer?

---

YOUR ANSWER IN THE CELL BELOW

YOUR ANSWER HERE

END OF YOUR ANSWER

---
2.5 _(1 point)_ Answer the following conceptual questions:

- What do you observe on the optimization trajectory when you change the learning rate? What do you see when we choose a learning rate that is too low or too high?
- How can you get a suitable learning rate for the Adam optimizer on your optimization problem?\

---

YOUR ANSWER IN THE CELL BELOW

YOUR ANSWER HERE

END OF YOUR ANSWER

---

2.5 _(2 points)_ Redo exercise 2.1 and 2.2 but know use the **gradient-free optimizer CMA-ES (Covariance Matrix Adaptation Evolution Strategy)**. Use the version implemented in exercise 1.

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

### Exercise 3

Initial conditions play a significant role in the behavior and outcome of optimization algorithms. The starting point of an optimization process can influence the trajectory taken by the optimizer and determine whether it converges to a global or local minimum.

In the `optimize_benchmark_fn` function, the initial starting point, $\vec{x}_0$, is generated by sampling from a random uniform distribution. By using different `seed` values, we can alter the random number generation, creating new starting points for the optimization process.

---

3.1 _(1 points)_ Repeat exercise 2.1 but now with a few different seeds: $123$, $124$, $125$ and $126$. Create two figures:
- Plot the objective value (y-axis) against the function evaluation count (x-axis) for each outcome in one figure.*
- Plot all the trajectories in one contour plot.

_\* You might want to use a logarithmic scale on the objective value to observe the differences more clearly_

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

3.2 _(1 point)_ Repeat exercise 3.1 but now with a different benchmark function the `'Branin'` for $2000$ iterations and $\vec{x} \in [-5.0, 15.0]$ function (more information about this function can be found [here](https://www.sfu.ca/~ssurjano/branin.html))

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

3.3 _(1 point)_ Repeat exercise 3.2 but now with the CMAES optimizer. Make sure you divide the number of iterations by the population size, since one iterations of CMAES creates multiple function evaluations!

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

3.3 _(1 point)_ Answer the following conceptual question:

- Can you explain in words what the influence is on the outcome of optimization if you are using different initial conditions on the two functions?

---

YOUR ANSWER IN THE CELL BELOW

YOUR ANSWER HERE

END OF YOUR ANSWER

---

End of the homework!